<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/save_pkl_into_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

将pkl的数据依次保存到对应名字的jsonl

In [1]:
%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 145 (delta 11), reused 16 (delta 7), pack-reused 115
Receiving objects: 100% (145/145), 36.81 MiB | 15.15 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/content/Embed-Adapter


In [2]:
!ls data

pseudo_inverses_400.pkl  pseudo_inverses.pkl  test_data.pkl


In [4]:

import pickle

# Load the pickle file
with open('/content/Embed-Adapter/data/pseudo_inverses.pkl', 'rb') as f:
    pseudo_inverses = pickle.load(f)

In [5]:
print(pseudo_inverses.keys())

dict_keys([('openai', 'BAAI/bge-small-zh-v1.5'), ('openai', 'BAAI/bge-base-zh-v1.5'), ('openai', 'BAAI/bge-large-zh-v1.5'), ('BAAI/bge-small-zh-v1.5', 'openai'), ('BAAI/bge-small-zh-v1.5', 'BAAI/bge-base-zh-v1.5'), ('BAAI/bge-small-zh-v1.5', 'BAAI/bge-large-zh-v1.5'), ('BAAI/bge-base-zh-v1.5', 'openai'), ('BAAI/bge-base-zh-v1.5', 'BAAI/bge-small-zh-v1.5'), ('BAAI/bge-base-zh-v1.5', 'BAAI/bge-large-zh-v1.5'), ('BAAI/bge-large-zh-v1.5', 'openai'), ('BAAI/bge-large-zh-v1.5', 'BAAI/bge-small-zh-v1.5'), ('BAAI/bge-large-zh-v1.5', 'BAAI/bge-base-zh-v1.5')])


In [8]:
print(pseudo_inverses[('openai', 'BAAI/bge-small-zh-v1.5')].shape)

(1536, 512)


In [38]:
for key in pseudo_inverses.keys():
    print(key, pseudo_inverses[key].shape)

('openai', 'BAAI/bge-small-zh-v1.5') (1536, 512)
('openai', 'BAAI/bge-base-zh-v1.5') (1536, 768)
('openai', 'BAAI/bge-large-zh-v1.5') (1536, 1024)
('BAAI/bge-small-zh-v1.5', 'openai') (512, 1536)
('BAAI/bge-small-zh-v1.5', 'BAAI/bge-base-zh-v1.5') (512, 768)
('BAAI/bge-small-zh-v1.5', 'BAAI/bge-large-zh-v1.5') (512, 1024)
('BAAI/bge-base-zh-v1.5', 'openai') (768, 1536)
('BAAI/bge-base-zh-v1.5', 'BAAI/bge-small-zh-v1.5') (768, 512)
('BAAI/bge-base-zh-v1.5', 'BAAI/bge-large-zh-v1.5') (768, 1024)
('BAAI/bge-large-zh-v1.5', 'openai') (1024, 1536)
('BAAI/bge-large-zh-v1.5', 'BAAI/bge-small-zh-v1.5') (1024, 512)
('BAAI/bge-large-zh-v1.5', 'BAAI/bge-base-zh-v1.5') (1024, 768)


In [27]:
!rm -rf /content/output

请为我实现以下python程序

save_folder = "/content/output"

先确认save_folder存在，如果不存在则新建这个目录

pseudo_inverses是一个dict，里面的的key都是(str1,str2)的形式，value都是np形式的矩阵

对于每个key，save_name = <str1>__<str2>.jsonl

如key是('openai', 'BAAI/bge-small-zh-v1.5')时，

把"/"替换为"---"

save_name = "openai__BAAI---bge-small-zh-v1.5.jsonl"

pseudo_inverses的value都是一个float的np array形式的矩阵

把pseudo_inverses[key]对应的矩阵存到save_data["matrix"]中

然后把save_data保存到save_name对应的jsonl文件中

In [28]:
pip install -q pandas pyarrow


In [30]:
import os
import pandas as pd
import numpy as np

save_folder = "/content/output"

# 确保保存目录存在
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# 假设pseudo_inverses是已经定义好的字典
# pseudo_inverses = ...

for key, matrix in pseudo_inverses.items():
    # 处理键以创建文件名
    str1, str2 = key
    str1 = str1.replace('/', '---')
    str2 = str2.replace('/', '---')
    save_name = f"{str1}__{str2}.parquet"
    save_path = os.path.join(save_folder, save_name)

    # 将矩阵转换为DataFrame，并指定列名
    column_names = [f"matrix_{i}" for i in range(matrix.shape[1])]
    df = pd.DataFrame(matrix, columns=column_names)

    # 保存数据到Parquet文件
    df.to_parquet(save_path)

print("数据保存完毕！")


数据保存完毕！


实现另一段python程序

把/content/output下的文件压缩到/content/inverses.zip

In [31]:
import os
import zipfile

# 指定源文件夹和目标ZIP文件的路径
source_folder = '/content/output'
zip_file_path = '/content/inverses.zip'

# 创建一个ZipFile对象以写入模式
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # 遍历source_folder中的所有文件
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            # 创建文件的完整路径
            file_path = os.path.join(root, file)
            # 将文件添加到zip文件中
            # arcname参数用于去除文件的源目录路径
            zipf.write(file_path, arcname=os.path.relpath(file_path, source_folder))

print("文件压缩完成！")


文件压缩完成！


In [ ]:
!pip install -q datasets

In [32]:


from datasets import load_dataset

dataset = load_dataset("silk-road/Embedding-Adapter",data_files={'train':"BAAI---bge-base-zh-v1.5__BAAI---bge-large-zh-v1.5.parquet"})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [50]:
import numpy as np
from tqdm import tqdm
def load_matrix( dataset ):
    n_row = len(dataset["train"])
    n_col = len(dataset["train"][0])
    matrix = np.zeros((n_row,n_col), dtype = float )
    for i in tqdm(range(n_row)):
        matrix[i] = list( dataset["train"][i].values())
        # for j in range(n_col):
        #     matrix[i,j] = dataset["train"][i]['matrix_'+ str(j)]
    return matrix

matrix = load_matrix(dataset)


100%|██████████| 768/768 [00:08<00:00, 88.00it/s]




我想验证这个载入后的matrix

和 矩阵 pseudo_inverses[('BAAI/bge-large-zh-v1.5', 'BAAI/bge-base-zh-v1.5')] 之间的差异，对于每个矩阵元素，如果有0.1%的元素误差，则count+1，统计这样的元素的个数

In [53]:
import numpy as np

# 假设 matrix 和 pseudo_inverse_matrix 已经被定义和加载
# matrix = load_matrix(dataset)
pseudo_inverse_matrix = pseudo_inverses[('BAAI/bge-base-zh-v1.5','BAAI/bge-large-zh-v1.5')]

# 计算两个矩阵之间的差异
difference = matrix - pseudo_inverse_matrix

# 计算相对误差，避免除以零
with np.errstate(divide='ignore', invalid='ignore'):
    relative_error = np.abs(difference / pseudo_inverse_matrix)
    relative_error[pseudo_inverse_matrix == 0] = 0  # 如果原始矩阵中的元素为0，则设置相对误差为0

# 统计误差超过0.1%的元素个数
error_threshold = 0.001  # 0.1%
count = np.sum(relative_error > error_threshold)

print(f"误差超过0.1%的元素个数: {count}")


误差超过0.1%的元素个数: 0


In [54]:

from datasets import load_dataset
import numpy as np
from tqdm import tqdm
def load_matrix( dataset ):
    n_row = len(dataset["train"])
    n_col = len(dataset["train"][0])
    matrix = np.zeros((n_row,n_col), dtype = float )
    for i in tqdm(range(n_row)):
        matrix[i] = list( dataset["train"][i].values())
    return matrix


class HFAdapter:
    def __init__(self, source_long_name, target_long_name, model_name = None):
        self.source_long_name = source_long_name
        self.target_long_name = target_long_name
        # 初始化任何必要的资源
        self.pesudo_inverse = self.get_pesudo_inverse(source_long_name, target_long_name, model_name)


    def get_pesudo_inverse( self,source_long_name, target_long_name , model_name = None):
        if model_name == None:
            str1 = source_long_name.replace("/","---")
            str2 = target_long_name.replace("/","---")
            model_name = str1 + "__" + str2 + ".parquet"
            dataset = load_dataset("silk-road/Embedding-Adapter",data_files={'train':model_name})
        else:
            print("try download adapte model from ", model_name , " but not yet implemented loading code")

        return load_matrix(dataset)

    def convert(self, input_embedding, output_format):
        # 这里实现转换逻辑
        if isinstance(input_embedding, list):
            input_embedding = np.array(input_embedding)

        # input_embedding is 1 * input_dim np array
        # self.pesudo_inverse is input_dim * output_dim np array
        # output_embedding is 1 * output_dim np array
        output_embedding = np.matmul(input_embedding, self.pesudo_inverse)

        if output_format == "np":
            return output_embedding
        elif output_format == "list":
            return output_embedding.tolist()
        else:
            raise ValueError("Unsupported output format")


In [55]:

%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 145 (delta 11), reused 16 (delta 7), pack-reused 115
Receiving objects: 100% (145/145), 36.81 MiB | 3.86 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/content/Embed-Adapter


In [57]:
from EmbedAdapter.utils import get_general_embedding

In [60]:

# 创建适配器实例
adapter = HFAdapter("BAAI/bge-small-zh-v1.5", "openai")

bge_embedding = get_general_embedding("我爱北京天安门", model_name = "BAAI/bge-small-zh-v1.5")
print(len(bge_embedding))

# 使用适配器转换embedding
openai_embedding = adapter.convert(bge_embedding,"list")

print(len(openai_embedding))

100%|██████████| 512/512 [00:08<00:00, 62.56it/s]

512
1536
